In [1]:
#Import Dependencies
import csv, sqlite3
import pandas as pd

In [2]:
#DATABASE name
database = '../data/project3db.sqlite'

In [3]:
db = sqlite3.connect(database)
cursor = db.cursor()
res = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print (name[0])
db.close()

Alc_test1
sqlite_sequence
HA_test_phone1
HA_test_watch1


In [4]:
#connect to the database
db = sqlite3.connect(database)

query = "SELECT * FROM Alc_test1;"
df = pd.read_sql_query(query,db)
db.close()
df.head()

,rec_id,FP1,FP2,F7,F8,AF1,AF2,FZ,F4,F3,...,PO8,FCZ,POZ,OZ,P2,P1,CPZ,nd,Y,Status
0,1,-42.582,-3.662,5.910,5.697,8.382,7.619,6.877,6.409,9.013,...,3.174,3.611,0.397,1.007,1.587,-2.299,-1.780,3.560,8.759,alcoholic
1,2,11.587,5.473,15.188,11.983,13.550,6.907,5.229,0.122,5.615,...,4.700,3.347,6.460,5.585,4.506,0.254,-0.214,17.385,18.473,control
2,3,-5.554,-8.077,-1.465,-2.136,-7.853,-6.989,-5.717,1.088,-5.656,...,-8.321,-1.221,-14.781,-14.191,-11.597,-7.935,-3.001,-8.006,-4.262,alcoholic
3,4,-3.296,-0.214,-0.326,5.910,-6.500,-2.736,-7.355,-6.480,-6.317,...,18.545,-5.086,8.565,10.010,8.097,8.596,6.053,8.474,8.341,alcoholic
4,5,-4.547,-3.174,-1.261,-6.042,-5.778,-4.639,-9.084,-3.082,-6.856,...,-11.902,-6.999,8.667,2.665,11.709,9.511,6.958,3.855,4.659,alcoholic


In [5]:
y_test = df["Status"]
y_test

0      alcoholic
1        control
2      alcoholic
3      alcoholic
4      alcoholic
5      alcoholic
6      alcoholic
7      alcoholic
8      alcoholic
9        control
10       control
11     alcoholic
12     alcoholic
13       control
14     alcoholic
15     alcoholic
16     alcoholic
17       control
18     alcoholic
19     alcoholic
20     alcoholic
21       control
22       control
23     alcoholic
24     alcoholic
25     alcoholic
26     alcoholic
27     alcoholic
28       control
29     alcoholic
         ...    
970      control
971    alcoholic
972    alcoholic
973    alcoholic
974    alcoholic
975    alcoholic
976    alcoholic
977      control
978    alcoholic
979      control
980    alcoholic
981      control
982      control
983    alcoholic
984    alcoholic
985    alcoholic
986    alcoholic
987    alcoholic
988    alcoholic
989    alcoholic
990    alcoholic
991    alcoholic
992    alcoholic
993      control
994    alcoholic
995    alcoholic
996    alcoholic
997    alcohol

In [7]:
X_test = df.iloc[:,1:65]
X_test.head()

,FP1,FP2,F7,F8,AF1,AF2,FZ,F4,F3,FC6,...,PO7,PO8,FCZ,POZ,OZ,P2,P1,CPZ,nd,Y
0,-42.582,-3.662,5.910,5.697,8.382,7.619,6.877,6.409,9.013,5.056,...,-2.472,3.174,3.611,0.397,1.007,1.587,-2.299,-1.780,3.560,8.759
1,11.587,5.473,15.188,11.983,13.550,6.907,5.229,0.122,5.615,2.543,...,4.415,4.700,3.347,6.460,5.585,4.506,0.254,-0.214,17.385,18.473
2,-5.554,-8.077,-1.465,-2.136,-7.853,-6.989,-5.717,1.088,-5.656,-1.027,...,-1.017,-8.321,-1.221,-14.781,-14.191,-11.597,-7.935,-3.001,-8.006,-4.262
3,-3.296,-0.214,-0.326,5.910,-6.500,-2.736,-7.355,-6.480,-6.317,-0.783,...,12.349,18.545,-5.086,8.565,10.010,8.097,8.596,6.053,8.474,8.341
4,-4.547,-3.174,-1.261,-6.042,-5.778,-4.639,-9.084,-3.082,-6.856,-1.170,...,0.661,-11.902,-6.999,8.667,2.665,11.709,9.511,6.958,3.855,4.659


In [8]:
#Load the Scaled function for LR model
from sklearn.externals.joblib import dump, load
X_scaler = load('Alc_Brain_LR_scaler.bin')

In [9]:
#Scales X_test data
X_test_scaled = X_scaler.transform(X_test)  

In [11]:
#Load Label Encoder
label_encoder = load('AB_NN_label_encoder.bin')

In [13]:
from keras.utils import to_categorical
encoded_y_test = label_encoder.transform(y_test)
y_test_categorical = to_categorical(encoded_y_test)

In [14]:
#Import a model (NN)
from keras.models import load_model
modeln1 = "ALC_NN_FDF1.h5"
model = load_model(modeln1)


In [15]:
#Evaluate accuracy of model with test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.17668986213207244, Accuracy: 0.921999990940094


In [16]:
#Example of predictions with model
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [17]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['alcoholic' 'control' 'alcoholic' 'alcoholic' 'alcoholic']
Actual Labels: ['alcoholic', 'control', 'alcoholic', 'alcoholic', 'alcoholic']


In [18]:
#Import second model (1CDNN)
modeln1 = "alcoholic_model_1DCNN1AlexData1.h5"
model = load_model(modeln1)

In [19]:
#Evaluate accuracy of model with test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.15482808071374893, Accuracy: 0.9359999895095825


In [20]:
#Example of predictions with model
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [21]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['alcoholic' 'control' 'alcoholic' 'alcoholic' 'control']
Actual Labels: ['alcoholic', 'control', 'alcoholic', 'alcoholic', 'alcoholic']
